# Task 1: Data Collection and Preprocessing for Fintech App Reviews

This notebook covers the first task of the Week 2 challenge: scraping user reviews from the Google Play Store for three Ethiopian banks (Commercial Bank of Ethiopia, Bank of Abyssinia, and Dashen Bank), followed by essential data preprocessing steps.

## 1. Setup and Library Imports

First, let's install and import the necessary libraries.
If you haven't already, ensure these are installed via your `requirements.txt` as we discussed earlier.

In [ ]:
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews_all
from datetime import datetime
import os 

print("Libraries imported successfully!")

## 2. Define App Package Names

We need the unique package names (app IDs) for each bank's mobile application on the Google Play Store. These were identified through prior search.

In [ ]:
app_packages = {
    'Commercial Bank of Ethiopia': 'com.combanketh.mobilebanking',
    'Bank of Abyssinia': 'com.boa.boaMobileBanking',
    'Dashen Bank': 'com.dashen.dashensuperapp'
}

print("App package names defined.")

## 3. Web Scraping Google Play Store Reviews

We will use the `google-play-scraper` library to collect reviews.
The goal is to collect at least 400 reviews per bank, totaling 1200+ reviews.
We will collect review text, rating, and date.

In [ ]:
all_reviews = []

for bank_name, app_id in app_packages.items():
    print(f"Scraping reviews for {bank_name} (App ID: {app_id})...")
    try:
        # Scrape reviews
        # Collecting more than 400 to ensure we have enough after cleaning
        result, continuation_token = reviews_all(
            app_id,
            lang='en', # English reviews
            country='et', # From Ethiopia
            sort=Sort.NEWEST, # Get the newest reviews
            count=1000 # Attempt to get up to 1000 reviews per app to ensure 400+ after cleaning
        )
        for r in result:
            all_reviews.append({
                'review_id': r['reviewId'],
                'bank': bank_name,
                'app_id': app_id,
                'review_text': r['content'],
                'rating': r['score'],
                'date': r['at'],
                'source': 'Google Play'
            })
        print(f"Successfully scraped {len(result)} reviews for {bank_name}.")
    except Exception as e:
        print(f"Error scraping {bank_name}: {e}")

# Create a DataFrame from the collected reviews
df_reviews = pd.DataFrame(all_reviews)
print(f"\nTotal raw reviews collected: {len(df_reviews)}")
print("Sample of raw data:")
print(df_reviews.head())

## 4. Data Preprocessing

This section handles data cleaning, including removing duplicates, handling missing data, and normalizing dates.

### 4.1 Remove Duplicates

Remove duplicate reviews based on `review_id` and `review_text`.

In [ ]:
df_reviews_cleaned = df_reviews.drop_duplicates(subset=['review_id', 'review_text']).copy()
print(f"\nReviews after removing duplicates: {len(df_reviews_cleaned)}")

### 4.2 Handle Missing Data

Check for missing values and remove/fill them as appropriate.

In [ ]:
print("\nMissing values before handling:")
print(df_reviews_cleaned.isnull().sum())

# Remove rows where `review_text` or `rating` is missing, as these are critical for analysis.
df_reviews_cleaned.dropna(subset=['review_text', 'rating'], inplace=True)
print(f"Reviews after dropping rows with missing critical data: {len(df_reviews_cleaned)}")

# Fill any other potential missing values if necessary (e.g., empty string for text, 'Unknown' for source)
df_reviews_cleaned['review_text'].fillna('', inplace=True)
df_reviews_cleaned['source'].fillna('Unknown', inplace=True)

print("\nMissing values after handling:")
print(df_reviews_cleaned.isnull().sum())

### 4.3 Normalize Dates

Convert the 'date' column to datetime objects and then format to `YYYY-MM-DD`.

In [ ]:
df_reviews_cleaned['date'] = pd.to_datetime(df_reviews_cleaned['date'])
df_reviews_cleaned['date'] = df_reviews_cleaned['date'].dt.strftime('%Y-%m-%d')
print("\nDates normalized to YYYY-MM-DD format.")
print("Sample of cleaned data dates:")
print(df_reviews_cleaned['date'].head())

### 4.4 Ensure Required Columns and Order

Ensure the final DataFrame has the required columns: `review`, `rating`, `date`, `bank`, `source`.
Rename 'review_text' to 'review' for consistency with requirements.

In [ ]:
df_reviews_cleaned.rename(columns={'review_text': 'review'}, inplace=True)

# Select and reorder columns
final_columns = ['review_id', 'review', 'rating', 'date', 'bank', 'source']
df_reviews_cleaned = df_reviews_cleaned[final_columns]

print("\nFinal cleaned DataFrame structure:")
print(df_reviews_cleaned.head())
print(f"\nTotal cleaned reviews: {len(df_reviews_cleaned)}")

# Check if we met the 1200+ reviews target
min_reviews_met = len(df_reviews_cleaned) >= 1200
print(f"\nMet minimum 1200 reviews target: {min_reviews_met}")
if not min_reviews_met:
    print("Warning: Did not meet the minimum 1200 reviews target. Consider increasing 'count' in scraping or reviewing app IDs.")

## 5. Save Cleaned Data

Save the cleaned DataFrame to a CSV file in the `data/processed/` directory.

In [ ]:
# Ensure the data/processed directory exists
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, 'google_play_reviews_cleaned.csv')
df_reviews_cleaned.to_csv(output_path, index=False)
print(f"\nCleaned data saved to {output_path}")

# Verify the saved file
try:
    df_check = pd.read_csv(output_path)
    print("\nVerification: Successfully loaded the saved CSV.")
    print(df_check.info())
except Exception as e:
    print(f"\nVerification failed: {e}")